In [2]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('data_to_use.csv')

## 부적합 데이터

In [3]:
def find_unsuitable_data(df):
    """STB_YN 컬럼에서 '부적합' 데이터를 찾는 함수"""
    
    # STB_YN이 붙은 컬럼들 찾기
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    print(f"STB_YN 컬럼들: {stb_columns}")
    
    # 부적합 데이터를 저장할 리스트
    unsuitable_data = []
    
    print("\n=== 부적합 데이터 상세 조회 ===")
    
    for col in stb_columns:
        # 해당 컬럼에서 '부적합' 값을 가진 행들 찾기
        unsuitable_mask = (df[col] == '부적합')
        unsuitable_rows = df[unsuitable_mask]
        
        print(f"\n{col}에서 부적합인 행: {len(unsuitable_rows)}건")
        
        if len(unsuitable_rows) > 0:
            # 대응하는 측정값 컬럼명
            value_col = col.replace('_STB_YN', '_RSLT_NMVL')
            
            for idx, row in unsuitable_rows.iterrows():
                print(f"\n--- {col} 부적합 데이터 ---")
                print(f"인덱스: {idx}")
                print(f"학교명: {row.get('SCHUL_NM', 'N/A')}")
                print(f"주소: {row.get('ADRCD_NM', 'N/A')}")
                print(f"학교종류코드: {row.get('SCHUL_KND_SC_CODE', 'N/A')}")
                print(f"학기: {row.get('SEM_STR', 'N/A')}")
                print(f"측정 결과: {row[col]}")
                
                if value_col in df.columns:
                    print(f"측정값: {row.get(value_col, 'N/A')}")
                
                # 부적합 데이터 리스트에 추가
                unsuitable_info = {
                    'index': idx,
                    'school_name': row.get('SCHUL_NM'),
                    'address': row.get('ADRCD_NM'),
                    'school_type_code': row.get('SCHUL_KND_SC_CODE'),
                    'semester': row.get('SEM_STR'),
                    'measurement_type': col,
                    'status': row[col],
                    'measurement_value': row.get(value_col) if value_col in df.columns else None
                }
                unsuitable_data.append(unsuitable_info)
    
    return unsuitable_data

def analyze_unsuitable_patterns(df, unsuitable_data):
    """부적합 데이터의 패턴 분석"""
    
    if not unsuitable_data:
        print("부적합 데이터가 없습니다.")
        return
    
    print(f"\n=== 부적합 데이터 패턴 분석 ===")
    print(f"총 부적합 건수: {len(unsuitable_data)}건")
    
    # DataFrame으로 변환
    unsuitable_df = pd.DataFrame(unsuitable_data)
    
    # 측정 항목별 부적합 건수
    print("\n1. 측정 항목별 부적합 건수:")
    item_counts = unsuitable_df['measurement_type'].value_counts()
    for item, count in item_counts.items():
        item_name = item.replace('_STB_YN', '')
        print(f"   {item_name}: {count}건")
    
    # 지역별 부적합 건수
    print("\n2. 지역별 부적합 건수:")
    if 'address' in unsuitable_df.columns:
        address_counts = unsuitable_df['address'].value_counts()
        for addr, count in address_counts.items():
            print(f"   {addr}: {count}건")
    
    # 학기별 부적합 건수
    print("\n3. 학기별 부적합 건수:")
    if 'semester' in unsuitable_df.columns:
        semester_counts = unsuitable_df['semester'].value_counts()
        for sem, count in semester_counts.items():
            print(f"   {sem}: {count}건")
    
    # 학교 유형별 부적합 건수
    print("\n4. 학교 유형별 부적합 건수:")
    if 'school_type_code' in unsuitable_df.columns:
        school_type_counts = unsuitable_df['school_type_code'].value_counts()
        for school_type, count in school_type_counts.items():
            school_level = get_school_level(school_type)
            print(f"   {school_level} (코드 {school_type}): {count}건")
    
    # 측정값 분석
    print("\n5. 부적합 측정값 분석:")
    for item_type in unsuitable_df['measurement_type'].unique():
        item_data = unsuitable_df[unsuitable_df['measurement_type'] == item_type]
        values = item_data['measurement_value'].dropna()
        
        if len(values) > 0:
            print(f"\n   {item_type.replace('_STB_YN', '')}:")
            print(f"     평균: {values.mean():.2f}")
            print(f"     최대: {values.max()}")
            print(f"     최소: {values.min()}")
            print(f"     측정값들: {list(values)}")
    
    return unsuitable_df

def get_school_level(code):
    """학교 종류 코드를 학교급으로 변환"""
    school_types = {
        2: '초등학교', 
        3: '중학교',
        4: '고등학교'
    }
    return school_types.get(code, f'기타({code})')

# 실행 코드
if __name__ == "__main__":
    print("=== 부적합 데이터 조회 및 분석 ===\n")
    
    # 1. 부적합 데이터 찾기
    unsuitable_data = find_unsuitable_data(df)
    
    # 2. 부적합 데이터 패턴 분석
    if unsuitable_data:
        unsuitable_df = analyze_unsuitable_patterns(df, unsuitable_data) 
    else:
        print("부적합 데이터가 발견되지 않았습니다.")

# 특정 조건으로 필터링하는 함수들
def filter_by_measurement_type(df, measurement_type):
    """특정 측정 항목의 부적합 데이터만 필터링"""
    stb_col = f"{measurement_type}_STB_YN"
    if stb_col in df.columns:
        return df[df[stb_col] == '부적합']
    else:
        print(f"{stb_col} 컬럼이 존재하지 않습니다.")
        return pd.DataFrame()

def filter_by_region(df, region):
    """특정 지역의 부적합 데이터만 필터링"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    # 모든 STB_YN 컬럼에서 부적합인 행들 찾기
    unsuitable_mask = pd.Series([False] * len(df))
    for col in stb_columns:
        unsuitable_mask |= (df[col] == '부적합')
    
    # 지역 필터링
    region_mask = df['ADRCD_NM'].str.contains(region, na=False)
    
    return df[unsuitable_mask & region_mask]

=== 부적합 데이터 조회 및 분석 ===

STB_YN 컬럼들: ['AIR_BACT_STB_YN', 'FALL_BACT_STB_YN', 'MITE_STB_YN', 'RN_STB_YN', 'O3_STB_YN', 'ASBESTOS_STB_YN']

=== 부적합 데이터 상세 조회 ===

AIR_BACT_STB_YN에서 부적합인 행: 0건

FALL_BACT_STB_YN에서 부적합인 행: 0건

MITE_STB_YN에서 부적합인 행: 0건

RN_STB_YN에서 부적합인 행: 0건

O3_STB_YN에서 부적합인 행: 3건

--- O3_STB_YN 부적합 데이터 ---
인덱스: 936
학교명: 서울금양초등학교
주소: 서울특별시 용산구
학교종류코드: 2
학기: 하반기
측정 결과: 부적합
측정값: 0.26

--- O3_STB_YN 부적합 데이터 ---
인덱스: 1667
학교명: 위례솔중학교
주소: 서울특별시 송파구
학교종류코드: 3
학기: 하반기
측정 결과: 부적합
측정값: 1.0

--- O3_STB_YN 부적합 데이터 ---
인덱스: 1763
학교명: 석관고등학교
주소: 서울특별시 성북구
학교종류코드: 4
학기: 하반기
측정 결과: 부적합
측정값: 0.8

ASBESTOS_STB_YN에서 부적합인 행: 0건

=== 부적합 데이터 패턴 분석 ===
총 부적합 건수: 3건

1. 측정 항목별 부적합 건수:
   O3: 3건

2. 지역별 부적합 건수:
   서울특별시 용산구: 1건
   서울특별시 송파구: 1건
   서울특별시 성북구: 1건

3. 학기별 부적합 건수:
   하반기: 3건

4. 학교 유형별 부적합 건수:
   초등학교 (코드 2): 1건
   중학교 (코드 3): 1건
   고등학교 (코드 4): 1건

5. 부적합 측정값 분석:

   O3:
     평균: 0.69
     최대: 1.0
     최소: 0.26
     측정값들: [0.26, 1.0, 0.8]


## 미실시 데이터

In [5]:
def find_not_implemented_data(df):
    """STB_YN 컬럼에서 '미실시' 데이터를 찾는 함수"""
    
    # STB_YN이 붙은 컬럼들 찾기
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    print(f"STB_YN 컬럼들: {stb_columns}")
    
    # 미실시 데이터를 저장할 리스트
    not_implemented_data = []
    
    print("\n=== 미실시 데이터 상세 조회 ===")
    
    for col in stb_columns:
        # 해당 컬럼에서 '미실시' 값을 가진 행들 찾기
        not_implemented_mask = (df[col] == '미실시')
        not_implemented_rows = df[not_implemented_mask]
        
        print(f"\n{col}에서 미실시인 행: {len(not_implemented_rows)}건")
        
        if len(not_implemented_rows) > 0:
            # 대응하는 측정값 컬럼명
            value_col = col.replace('_STB_YN', '_RSLT_NMVL')
            
            for idx, row in not_implemented_rows.iterrows():
                print(f"\n--- {col} 미실시 데이터 ---")
                print(f"인덱스: {idx}")
                print(f"학교명: {row.get('SCHUL_NM', 'N/A')}")
                print(f"주소: {row.get('ADRCD_NM', 'N/A')}")
                print(f"학교종류코드: {row.get('SCHUL_KND_SC_CODE', 'N/A')}")
                print(f"학기: {row.get('SEM_STR', 'N/A')}")
                print(f"측정 결과: {row[col]}")
                
                if value_col in df.columns:
                    print(f"측정값: {row.get(value_col, 'N/A')}")
                
                # 미실시 데이터 리스트에 추가
                not_implemented_info = {
                    'index': idx,
                    'school_name': row.get('SCHUL_NM'),
                    'address': row.get('ADRCD_NM'),
                    'school_type_code': row.get('SCHUL_KND_SC_CODE'),
                    'semester': row.get('SEM_STR'),
                    'measurement_type': col,
                    'status': row[col],
                    'measurement_value': row.get(value_col) if value_col in df.columns else None
                }
                not_implemented_data.append(not_implemented_info)
    
    return not_implemented_data

def analyze_not_implemented_patterns(df, not_implemented_data):
    """미실시 데이터의 패턴 분석"""
    
    if not not_implemented_data:
        print("미실시 데이터가 없습니다.")
        return
    
    print(f"\n=== 미실시 데이터 패턴 분석 ===")
    print(f"총 미실시 건수: {len(not_implemented_data)}건")
    
    # DataFrame으로 변환
    not_implemented_df = pd.DataFrame(not_implemented_data)
    
    # 측정 항목별 미실시 건수
    print("\n1. 측정 항목별 미실시 건수:")
    item_counts = not_implemented_df['measurement_type'].value_counts()
    for item, count in item_counts.items():
        item_name = item.replace('_STB_YN', '')
        print(f"   {item_name}: {count}건")
    
    # 지역별 미실시 건수
    print("\n2. 지역별 미실시 건수:")
    if 'address' in not_implemented_df.columns:
        address_counts = not_implemented_df['address'].value_counts()
        for addr, count in address_counts.items():
            print(f"   {addr}: {count}건")
    
    # 학기별 미실시 건수
    print("\n3. 학기별 미실시 건수:")
    if 'semester' in not_implemented_df.columns:
        semester_counts = not_implemented_df['semester'].value_counts()
        for sem, count in semester_counts.items():
            print(f"   {sem}: {count}건")
    
    # 학교 유형별 미실시 건수
    print("\n4. 학교 유형별 미실시 건수:")
    if 'school_type_code' in not_implemented_df.columns:
        school_type_counts = not_implemented_df['school_type_code'].value_counts()
        for school_type, count in school_type_counts.items():
            school_level = get_school_level(school_type)
            print(f"   {school_level} (코드 {school_type}): {count}건")
    
    # 측정값 분석 (미실시인 경우 측정값이 있는지 확인)
    print("\n5. 미실시 데이터의 측정값 현황:")
    for item_type in not_implemented_df['measurement_type'].unique():
        item_data = not_implemented_df[not_implemented_df['measurement_type'] == item_type]
        values = item_data['measurement_value'].dropna()
        
        print(f"\n   {item_type.replace('_STB_YN', '')}:")
        print(f"     미실시 건수: {len(item_data)}건")
        print(f"     측정값이 있는 건수: {len(values)}건")
        print(f"     측정값이 없는 건수: {len(item_data) - len(values)}건")
        
        if len(values) > 0:
            print(f"     측정값 범위: {values.min()} ~ {values.max()}")
            print(f"     측정값들: {list(values)}")
    
    return not_implemented_df

def get_school_level(code):
    """학교 종류 코드를 학교급으로 변환"""
    school_types = {
        2: '초등학교', 
        3: '중학교',
        4: '고등학교'
    }
    return school_types.get(code, f'기타({code})')

def create_not_implemented_summary_table(not_implemented_df):
    """미실시 데이터 요약 테이블 생성"""
    
    if not_implemented_df is None or len(not_implemented_df) == 0:
        print("미실시 데이터가 없어 요약 테이블을 생성할 수 없습니다.")
        return None
    
    print("\n=== 미실시 데이터 요약 테이블 ===")
    
    # 요약 테이블 생성
    summary_table = not_implemented_df.groupby(['measurement_type', 'address']).agg({
        'school_name': 'count',
        'measurement_value': 'count'  # 측정값이 있는 건수
    })
    
    summary_table.columns = ['총 미실시 건수', '측정값 있는 건수']
    
    print(summary_table)
    
    return summary_table

# 실행 코드
if __name__ == "__main__":
    print("=== 미실시 데이터 조회 및 분석 ===\n")
    
    # 1. 미실시 데이터 찾기
    not_implemented_data = find_not_implemented_data(df)
    
    # 2. 미실시 데이터 패턴 분석
    if not_implemented_data:
        not_implemented_df = analyze_not_implemented_patterns(df, not_implemented_data)
        
        # 3. 요약 테이블 생성
        summary_table = create_not_implemented_summary_table(not_implemented_df)        
    else:
        print("미실시 데이터가 발견되지 않았습니다.")

# 특정 조건으로 필터링하는 함수들
def filter_by_measurement_type_not_implemented(df, measurement_type):
    """특정 측정 항목의 미실시 데이터만 필터링"""
    stb_col = f"{measurement_type}_STB_YN"
    if stb_col in df.columns:
        return df[df[stb_col] == '미실시']
    else:
        print(f"{stb_col} 컬럼이 존재하지 않습니다.")
        return pd.DataFrame()

def filter_by_region_not_implemented(df, region):
    """특정 지역의 미실시 데이터만 필터링"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    # 모든 STB_YN 컬럼에서 미실시인 행들 찾기
    not_implemented_mask = pd.Series([False] * len(df))
    for col in stb_columns:
        not_implemented_mask |= (df[col] == '미실시')
    
    # 지역 필터링
    region_mask = df['ADRCD_NM'].str.contains(region, na=False)
    
    return df[not_implemented_mask & region_mask]

def analyze_not_implemented_by_semester(df):
    """학기별 미실시 현황 분석"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    print("\n=== 학기별 미실시 현황 분석 ===")
    
    for semester in df['SEM_STR'].unique():
        if pd.isna(semester):
            continue
            
        semester_data = df[df['SEM_STR'] == semester]
        print(f"\n{semester}:")
        
        for col in stb_columns:
            not_implemented_count = (semester_data[col] == '미실시').sum()
            total_count = len(semester_data)
            percentage = (not_implemented_count / total_count * 100) if total_count > 0 else 0
            
            item_name = col.replace('_STB_YN', '')
            print(f"  {item_name}: {not_implemented_count}건 ({percentage:.1f}%)")

def analyze_not_implemented_by_school_type(df):
    """학교 유형별 미실시 현황 분석"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    print("\n=== 학교 유형별 미실시 현황 분석 ===")
    
    for school_type in df['SCHUL_KND_SC_CODE'].unique():
        if pd.isna(school_type):
            continue
            
        school_type_data = df[df['SCHUL_KND_SC_CODE'] == school_type]
        school_level = get_school_level(school_type)
        
        print(f"\n{school_level} (코드 {school_type}):")
        
        for col in stb_columns:
            not_implemented_count = (school_type_data[col] == '미실시').sum()
            total_count = len(school_type_data)
            percentage = (not_implemented_count / total_count * 100) if total_count > 0 else 0
            
            item_name = col.replace('_STB_YN', '')
            print(f"  {item_name}: {not_implemented_count}건 ({percentage:.1f}%)")

=== 미실시 데이터 조회 및 분석 ===

STB_YN 컬럼들: ['AIR_BACT_STB_YN', 'FALL_BACT_STB_YN', 'MITE_STB_YN', 'RN_STB_YN', 'O3_STB_YN', 'ASBESTOS_STB_YN']

=== 미실시 데이터 상세 조회 ===

AIR_BACT_STB_YN에서 미실시인 행: 30건

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 22
학교명: 서울대청초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 하반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 23
학교명: 서울대치초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 상반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 89
학교명: 서울잠원초등학교
주소: 서울특별시 서초구
학교종류코드: 2
학기: 상반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 107
학교명: 서울강동초등학교
주소: 서울특별시 강동구
학교종류코드: 2
학기: 상반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 139
학교명: 서울문정초등학교
주소: 서울특별시 송파구
학교종류코드: 2
학기: 상반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 148
학교명: 서울상일초등학교
주소: 서울특별시 강동구
학교종류코드: 2
학기: 하반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 159
학교명: 서울세륜초등학교
주소: 서울특별시 송파구
학교종류코드: 2
학기: 상반기
측정 결과: 미실시
측정값: nan

--- AIR_BACT_STB_YN 미실시 데이터 ---
인덱스: 160
학교명: 서울세륜초등학교


## 해당없음

In [6]:
def find_not_applicable_data(df):
    """STB_YN 컬럼에서 '해당없음' 데이터를 찾는 함수"""
    
    # STB_YN이 붙은 컬럼들 찾기
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    print(f"STB_YN 컬럼들: {stb_columns}")
    
    # 해당없음 데이터를 저장할 리스트
    not_applicable_data = []
    
    print("\n=== 해당없음 데이터 상세 조회 ===")
    
    for col in stb_columns:
        # 해당 컬럼에서 '해당없음' 값을 가진 행들 찾기
        not_applicable_mask = (df[col] == '해당없음')
        not_applicable_rows = df[not_applicable_mask]
        
        print(f"\n{col}에서 해당없음인 행: {len(not_applicable_rows)}건")
        
        if len(not_applicable_rows) > 0:
            # 대응하는 측정값 컬럼명
            value_col = col.replace('_STB_YN', '_RSLT_NMVL')
            
            # 처음 5개만 출력 (데이터가 많을 수 있으므로)
            display_count = min(5, len(not_applicable_rows))
            print(f"  (처음 {display_count}건만 표시)")
            
            for i, (idx, row) in enumerate(not_applicable_rows.iterrows()):
                if i >= display_count:
                    break
                    
                print(f"\n--- {col} 해당없음 데이터 {i+1} ---")
                print(f"인덱스: {idx}")
                print(f"학교명: {row.get('SCHUL_NM', 'N/A')}")
                print(f"주소: {row.get('ADRCD_NM', 'N/A')}")
                print(f"학교종류코드: {row.get('SCHUL_KND_SC_CODE', 'N/A')}")
                print(f"학기: {row.get('SEM_STR', 'N/A')}")
                print(f"측정 결과: {row[col]}")
                
                if value_col in df.columns:
                    print(f"측정값: {row.get(value_col, 'N/A')}")
            
            # 모든 해당없음 데이터를 리스트에 추가
            for idx, row in not_applicable_rows.iterrows():
                not_applicable_info = {
                    'index': idx,
                    'school_name': row.get('SCHUL_NM'),
                    'address': row.get('ADRCD_NM'),
                    'school_type_code': row.get('SCHUL_KND_SC_CODE'),
                    'semester': row.get('SEM_STR'),
                    'measurement_type': col,
                    'status': row[col],
                    'measurement_value': row.get(value_col) if value_col in df.columns else None
                }
                not_applicable_data.append(not_applicable_info)
    
    return not_applicable_data

def analyze_not_applicable_patterns(df, not_applicable_data):
    """해당없음 데이터의 패턴 분석"""
    
    if not not_applicable_data:
        print("해당없음 데이터가 없습니다.")
        return
    
    print(f"\n=== 해당없음 데이터 패턴 분석 ===")
    print(f"총 해당없음 건수: {len(not_applicable_data)}건")
    
    # DataFrame으로 변환
    not_applicable_df = pd.DataFrame(not_applicable_data)
    
    # 측정 항목별 해당없음 건수
    print("\n1. 측정 항목별 해당없음 건수:")
    item_counts = not_applicable_df['measurement_type'].value_counts()
    for item, count in item_counts.items():
        item_name = item.replace('_STB_YN', '')
        total_schools = len(df)
        percentage = (count / total_schools * 100) if total_schools > 0 else 0
        print(f"   {item_name}: {count}건 ({percentage:.1f}%)")
    
    # 지역별 해당없음 건수 (상위 10개 지역만)
    print("\n2. 지역별 해당없음 건수 (상위 10개):")
    if 'address' in not_applicable_df.columns:
        address_counts = not_applicable_df['address'].value_counts().head(10)
        for addr, count in address_counts.items():
            print(f"   {addr}: {count}건")
    
    # 학기별 해당없음 건수
    print("\n3. 학기별 해당없음 건수:")
    if 'semester' in not_applicable_df.columns:
        semester_counts = not_applicable_df['semester'].value_counts()
        for sem, count in semester_counts.items():
            print(f"   {sem}: {count}건")
    
    # 학교 유형별 해당없음 건수
    print("\n4. 학교 유형별 해당없음 건수:")
    if 'school_type_code' in not_applicable_df.columns:
        school_type_counts = not_applicable_df['school_type_code'].value_counts()
        for school_type, count in school_type_counts.items():
            school_level = get_school_level(school_type)
            print(f"   {school_level} (코드 {school_type}): {count}건")
    
    # 측정값 분석 (해당없음인 경우 측정값이 있는지 확인)
    print("\n5. 해당없음 데이터의 측정값 현황:")
    for item_type in not_applicable_df['measurement_type'].unique():
        item_data = not_applicable_df[not_applicable_df['measurement_type'] == item_type]
        values = item_data['measurement_value'].dropna()
        
        print(f"\n   {item_type.replace('_STB_YN', '')}:")
        print(f"     해당없음 건수: {len(item_data)}건")
        print(f"     측정값이 있는 건수: {len(values)}건")
        print(f"     측정값이 없는 건수: {len(item_data) - len(values)}건")
        
        if len(values) > 0:
            print(f"     측정값 범위: {values.min()} ~ {values.max()}")
            # 측정값이 많을 수 있으므로 요약 통계만 표시
            print(f"     평균 측정값: {values.mean():.2f}")
    
    return not_applicable_df

def get_school_level(code):
    """학교 종류 코드를 학교급으로 변환"""
    school_types = {
        2: '초등학교', 
        3: '중학교',
        4: '고등학교'
    }
    return school_types.get(code, f'기타({code})')

def analyze_not_applicable_reasons(df):
    """해당없음 처리 사유 분석 (측정 항목별 특성 파악)"""
    
    print("\n=== 해당없음 처리 사유 분석 ===")
    
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    # 각 측정 항목별로 해당없음 비율 계산
    print("\n측정 항목별 해당없음 비율:")
    for col in stb_columns:
        total_count = len(df)
        not_applicable_count = (df[col] == '해당없음').sum()
        suitable_count = (df[col] == '적합').sum()
        not_implemented_count = (df[col] == '미실시').sum()
        unsuitable_count = (df[col] == '부적합').sum()
        
        item_name = col.replace('_STB_YN', '')
        
        print(f"\n{item_name}:")
        print(f"  해당없음: {not_applicable_count}건 ({not_applicable_count/total_count*100:.1f}%)")
        print(f"  적합: {suitable_count}건 ({suitable_count/total_count*100:.1f}%)")
        print(f"  미실시: {not_implemented_count}건 ({not_implemented_count/total_count*100:.1f}%)")
        print(f"  부적합: {unsuitable_count}건 ({unsuitable_count/total_count*100:.1f}%)")

def create_not_applicable_summary_table(not_applicable_df):
    """해당없음 데이터 요약 테이블 생성"""
    
    if not_applicable_df is None or len(not_applicable_df) == 0:
        print("해당없음 데이터가 없어 요약 테이블을 생성할 수 없습니다.")
        return None
    
    print("\n=== 해당없음 데이터 요약 테이블 ===")
    
    # 요약 테이블 생성
    summary_table = not_applicable_df.groupby(['measurement_type']).agg({
        'school_name': 'count',
        'measurement_value': 'count'  # 측정값이 있는 건수
    })
    
    summary_table.columns = ['총 해당없음 건수', '측정값 있는 건수']
    summary_table.index = [idx.replace('_STB_YN', '') for idx in summary_table.index]
    
    print(summary_table)
    
    return summary_table

# 실행 코드
if __name__ == "__main__":
    print("=== 해당없음 데이터 조회 및 분석 ===\n")
    
    # 1. 해당없음 데이터 찾기
    not_applicable_data = find_not_applicable_data(df)
    
    # 2. 해당없음 데이터 패턴 분석
    if not_applicable_data:
        not_applicable_df = analyze_not_applicable_patterns(df, not_applicable_data)
        
        # 3. 해당없음 처리 사유 분석
        analyze_not_applicable_reasons(df)
        
        # 4. 요약 테이블 생성
        summary_table = create_not_applicable_summary_table(not_applicable_df)        
    else:
        print("해당없음 데이터가 발견되지 않았습니다.")

# 특정 조건으로 필터링하는 함수들
def filter_by_measurement_type_not_applicable(df, measurement_type):
    """특정 측정 항목의 해당없음 데이터만 필터링"""
    stb_col = f"{measurement_type}_STB_YN"
    if stb_col in df.columns:
        return df[df[stb_col] == '해당없음']
    else:
        print(f"{stb_col} 컬럼이 존재하지 않습니다.")
        return pd.DataFrame()

def filter_by_region_not_applicable(df, region):
    """특정 지역의 해당없음 데이터만 필터링"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    # 모든 STB_YN 컬럼에서 해당없음인 행들 찾기
    not_applicable_mask = pd.Series([False] * len(df))
    for col in stb_columns:
        not_applicable_mask |= (df[col] == '해당없음')
    
    # 지역 필터링
    region_mask = df['ADRCD_NM'].str.contains(region, na=False)
    
    return df[not_applicable_mask & region_mask]

def analyze_not_applicable_by_semester(df):
    """학기별 해당없음 현황 분석"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    print("\n=== 학기별 해당없음 현황 분석 ===")
    
    for semester in df['SEM_STR'].unique():
        if pd.isna(semester):
            continue
            
        semester_data = df[df['SEM_STR'] == semester]
        print(f"\n{semester}:")
        
        for col in stb_columns:
            not_applicable_count = (semester_data[col] == '해당없음').sum()
            total_count = len(semester_data)
            percentage = (not_applicable_count / total_count * 100) if total_count > 0 else 0
            
            item_name = col.replace('_STB_YN', '')
            print(f"  {item_name}: {not_applicable_count}건 ({percentage:.1f}%)")

def analyze_not_applicable_by_school_type(df):
    """학교 유형별 해당없음 현황 분석"""
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    print("\n=== 학교 유형별 해당없음 현황 분석 ===")
    
    for school_type in df['SCHUL_KND_SC_CODE'].unique():
        if pd.isna(school_type):
            continue
            
        school_type_data = df[df['SCHUL_KND_SC_CODE'] == school_type]
        school_level = get_school_level(school_type)
        
        print(f"\n{school_level} (코드 {school_type}):")
        
        for col in stb_columns:
            not_applicable_count = (school_type_data[col] == '해당없음').sum()
            total_count = len(school_type_data)
            percentage = (not_applicable_count / total_count * 100) if total_count > 0 else 0
            
            item_name = col.replace('_STB_YN', '')
            print(f"  {item_name}: {not_applicable_count}건 ({percentage:.1f}%)")

def compare_measurement_necessity(df):
    """측정 필요성 비교 분석 (해당없음 vs 기타 상태)"""
    
    print("\n=== 측정 필요성 비교 분석 ===")
    
    stb_columns = [col for col in df.columns if col.endswith('STB_YN')]
    
    for col in stb_columns:
        item_name = col.replace('_STB_YN', '')
        
        # 각 상태별 건수 계산
        status_counts = df[col].value_counts()
        total_count = len(df)
        
        print(f"\n{item_name} 측정 현황:")
        
        # 해당없음 비율이 높은 경우 특별히 표시
        not_applicable_count = status_counts.get('해당없음', 0)
        not_applicable_rate = (not_applicable_count / total_count * 100) if total_count > 0 else 0
        
        for status, count in status_counts.items():
            rate = (count / total_count * 100) if total_count > 0 else 0
            marker = " ⚠️" if status == '해당없음' and rate > 20 else ""
            print(f"  {status}: {count}건 ({rate:.1f}%){marker}")
        
        # 해당없음 비율이 높은 경우 설명
        if not_applicable_rate > 20:
            print(f"  → {item_name}은 많은 학교에서 해당없음 처리됨 (측정 필요성 재검토 권장)")

=== 해당없음 데이터 조회 및 분석 ===

STB_YN 컬럼들: ['AIR_BACT_STB_YN', 'FALL_BACT_STB_YN', 'MITE_STB_YN', 'RN_STB_YN', 'O3_STB_YN', 'ASBESTOS_STB_YN']

=== 해당없음 데이터 상세 조회 ===

AIR_BACT_STB_YN에서 해당없음인 행: 0건

FALL_BACT_STB_YN에서 해당없음인 행: 550건
  (처음 5건만 표시)

--- FALL_BACT_STB_YN 해당없음 데이터 1 ---
인덱스: 0
학교명: 서울교육대학교부설초등학교
주소: 서울특별시 서초구
학교종류코드: 2
학기: 상반기
측정 결과: 해당없음
측정값: nan

--- FALL_BACT_STB_YN 해당없음 데이터 2 ---
인덱스: 4
학교명: 서울개일초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 상반기
측정 결과: 해당없음
측정값: nan

--- FALL_BACT_STB_YN 해당없음 데이터 3 ---
인덱스: 5
학교명: 서울개일초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 하반기
측정 결과: 해당없음
측정값: nan

--- FALL_BACT_STB_YN 해당없음 데이터 4 ---
인덱스: 7
학교명: 서울구룡초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 상반기
측정 결과: 해당없음
측정값: nan

--- FALL_BACT_STB_YN 해당없음 데이터 5 ---
인덱스: 10
학교명: 서울논현초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 하반기
측정 결과: 해당없음
측정값: nan

MITE_STB_YN에서 해당없음인 행: 45건
  (처음 5건만 표시)

--- MITE_STB_YN 해당없음 데이터 1 ---
인덱스: 12
학교명: 서울대곡초등학교
주소: 서울특별시 강남구
학교종류코드: 2
학기: 하반기
측정 결과: 해당없음
측정값: nan

--- MITE_STB_YN 해당없음 데이터 2 ---
인덱스: 18
학교명: 서울대왕